In [0]:
pip install azure-storage-blob

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-d09a4165-6acb-42bc-b9f0-ba7c72f21172
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
import requests
import pandas as pd
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.pipeline.transport import HttpResponse
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
import gzip
import plotly
import plotly.graph_objects as go
from pyspark.sql.functions import dayofmonth, countDistinct, to_date, hour


In [0]:
spark = SparkSession.builder.getOrCreate()

storage_account_name = "pod4projectstorage"
storage_account_access_key = "2hClDrVPLGX4QBDBk8OylAkHqczIQfDja66Yl488rmj/0+vb+CAzOxL5qMe5XyM9ZupgwveVRm3N+AStriO5vg=="

container_name = "pod4-project-blob-container"
directory_path = ""

spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)

df = spark.read.parquet("wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/" + directory_path)


In [0]:
df.show()

+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+---------------------+--------------------+--------------------+---------------+-------------------+--------------------+------------+----------------+
|                type|public|          created_at|                 org|               actor|                repo|payload.action|      payload.before|     payload.commits|payload.repository_id|      payload.review|       payload.pages|payload.push_id|payload.pusher_type|         payload.ref|payload.size|payload.ref_type|
+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+---------------------+--------------------+--------------------+---------------+-------------------+--------------------+------------+----------------+
|          WatchEvent|  true|2023-

In [0]:
grouped_df = df.groupBy("type").count().orderBy("count", ascending=False)

In [0]:
pd_data = grouped_df.toPandas()
pd_data

,type,count
0,PushEvent,62375369
1,CreateEvent,15508107
2,PullRequestEvent,8610684
3,WatchEvent,6525132
4,IssueCommentEvent,5170142
5,DeleteEvent,3540874
6,PullRequestReviewEvent,2834574
7,IssuesEvent,2366505
8,PullRequestReviewCommentEvent,1685195
9,ForkEvent,1555745


In [0]:
fig = go.Figure([go.Bar(x=pd_data['type'], y=pd_data['count'],marker=dict(color='blue'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Event Counts',xaxis=dict(tickangle=90))
fig.show()

In [0]:
push_df = df.filter(df.type == "PushEvent")

In [0]:
hour_df = push_df.withColumn('hour', hour(push_df['created_at']))

In [0]:
hour_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+---------------------+--------------+-------------+---------------+-------------------+--------------------+------------+----------------+----+
|     type|public|          created_at|                 org|               actor|                repo|payload.action|      payload.before|     payload.commits|payload.repository_id|payload.review|payload.pages|payload.push_id|payload.pusher_type|         payload.ref|payload.size|payload.ref_type|hour|
+---------+------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+---------------------+--------------+-------------+---------------+-------------------+--------------------+------------+----------------+----+
|PushEvent|  true|2023-05-01T00:00:00Z|                null|{https://avatars....|{622742349

In [0]:
grouped_hour = hour_df.groupBy("hour").count().orderBy("count", ascending=False)

In [0]:
pd_hour = grouped_hour.toPandas()

In [0]:
fig = go.Figure([go.Bar(x=pd_hour['hour'], y=pd_hour['count'],marker=dict(color='orangered'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig.update_layout(title_text='Commits by Hour',width = 1000, height = 550)
fig.show()

In [0]:
result = push_df.groupBy(dayofmonth("created_at").alias("Day")).agg(countDistinct("actor.id").alias("Users")).orderBy("Users", ascending=False)

In [0]:
result.show()

+---+------+
|Day| Users|
+---+------+
| 16|424008|
| 24|423598|
| 23|422278|
| 10|418778|
|  4|416232|
| 31|414916|
|  9|413888|
| 11|413709|
| 17|413524|
| 25|413127|
| 30|412549|
|  3|411701|
| 22|409983|
| 15|408045|
|  2|407199|
|  8|405626|
| 18|403107|
|  5|391737|
| 26|390119|
| 12|390003|
+---+------+
only showing top 20 rows



In [0]:
pd_date = result.toPandas()

In [0]:
fig = go.Figure([go.Bar(x=pd_date['Day'], y=pd_date['Users'], marker=dict(
        color=['yellowgreen' if val == pd_date['Users'].max() else 'royalblue' for val in pd_date['Users']]))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig.update_layout(title_text='Commits by Day of the Month',width = 1000, height = 550)
fig.show()

In [0]:
grouped_pushdf = push_df.select('org.login').dropna()

In [0]:
grouped_pushdf = grouped_pushdf.groupBy("login").count().orderBy("count", ascending=False)

In [0]:
grouped_pushdf.show()

+--------------------+------+
|               login| count|
+--------------------+------+
|          tf-vcs-e2e|188786|
|     networkoperator|161876|
|           newstools| 86995|
|         isomerpages| 74550|
|            Risk-DAO| 73382|
| redhat-appstudio-qe| 69836|
|        magicwave-io| 68005|
|         feedarchive| 54788|
|Ultimate-Hosts-Bl...| 46156|
|  MetadataGitTesting| 45595|
|            acciojob| 37684|
|redhat-appstudio-...| 36511|
|           microsoft| 31454|
|               hmcts| 31333|
|             unifyai| 30700|
|                odoo| 30568|
|     ecomplus-stores| 29155|
|    scalr-automation| 25363|
|           rust-lang| 25294|
|          factcheckr| 23613|
+--------------------+------+
only showing top 20 rows



In [0]:
pd_pushdf = grouped_pushdf.toPandas()
pd_pushdf = pd_pushdf.head()

In [0]:
pd_pushdf

,login,count
0,tf-vcs-e2e,188786
1,networkoperator,161876
2,newstools,86995
3,isomerpages,74550
4,Risk-DAO,73382


In [0]:
fig = go.Figure([go.Bar(x=pd_pushdf['login'], y=pd_pushdf['count'],marker=dict(color='yellowgreen'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.5)
fig.update_layout(title_text='Organizations with highest number of commits',width = 1000, height = 550)
fig.show()